In [1]:
! pip install neptune-client

     |████████████████████████████████| 315 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 4.5 MB/s  eta 0:00:01
  Created wheel for neptune-client: filename=neptune_client-0.15.2-py2.py3-none-any.whl size=565172 sha256=e0d809a657d154e8eeaeba1f1eee84191dcc7526fdbbc76262cba76026972dfc
  Stored in directory: /root/.cache/pip/wheels/4f/ff/e6/2128c7fd2e85bd327f524d44b07922848c40e48de6e75baa7e
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-py3-none-any.whl size=18149 sha256=ca2b92f1b957b730a06af1232c56ac6a5665a6747b6817946aaa1e80de96f00d
  Stored in directory: /root/.cache/pip/wheels/f3/1d/9f/2a74caecb81b8beb9a4fbe1754203d4b7cf42ef5d39e0d2311
Successfully built neptune-client strict-rfc3339


In [2]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

curl: /opt/conda/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6034  100  6034    0     0  24781      0 --:--:-- --:--:-- --:--:-- 24831
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Found existing installation: torch 1.7.1+cpu
Uninstalling torch-1.7.1+cpu:
  Successfully uninstalled torch-1.7.1+cpu
Found existing installation: torchvision 0.8.2+cpu
Uninstalling torchvision-0.8.2+cpu:
  Successfully uninstalled torchvision-0.8.2+cpu
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl...
- [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200515-cp37-cp37m-linux_x86_6

In [3]:
import glob
import tensorflow as tf
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import albumentations
from PIL import Image
import io
import matplotlib.pyplot as plt
import neptune.new as neptune
import torch.optim as optim
import torchvision.models as models
import numpy as np
import torch_xla
import torch_xla.core.xla_model as xm

2022-03-18 20:37:11.949186: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-03-18 20:37:11.949303: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[('__call__', <function LevelMapper.__call__ at 0x7fe4e5cea950>), ('__init__', <function LevelMapper.__init__ at 0x7fe4e5cea8c0>)]
[('__call__', <function BalancedPositiveNegativeSampler.__call__ at 0x7fe4e5bb93b0>), ('__init__', <function BalancedPositiveNegativeSampler.__init__ at 0x7fe4e5bb9320>)]
[('__init__', <function BoxCoder.__init__ at 0x7fe4e5bc0a70>), ('decode', <function BoxCoder.decode at 0x7fe4e5bc0c20>), ('decode_single', <function BoxCoder.decode_single at 0x7fe4e5bc0cb0>), ('encode', <function BoxCoder.encode at 0x7fe4e5bc0b00>), ('encode_single', <function BoxCoder.encode_single at 0x7fe4e5bc0b90>)]
[('__call__', <function Matcher.__call__ at 0x7fe4e5cb9290>), ('__init__', <function Matcher.__init__ at 0x7fe4e5cb9200>), ('set_low_quality_matches_', <function Matcher.set_low_quality_matches_ at 0x7fe4e5bc0dd0>)]
[('__init__', <function ImageList.__init__ at 0x7fe4e5bc08c0>), ('to', <function ImageList.to at 0x7fe4e5bc0ef0>)]
[('__init__', <function Timebase.__init__ at

In [4]:
train_files = glob.glob('../input/tpu-getting-started/tfrecords-jpeg-224x224/train/*.tfrec')
val_files = glob.glob('../input/tpu-getting-started/tfrecords-jpeg-224x224/val/*.tfrec')

In [5]:
feature_description = {
    'id': tf.io.FixedLenFeature([], tf.string),
    'class': tf.io.FixedLenFeature([], tf.int64),
    'image': tf.io.FixedLenFeature([], tf.string)
}



In [6]:
class PetalDataset(Dataset):
    def __init__(self,image_files):
        self.image_files = image_files
        self.features = {
            'id': tf.io.FixedLenFeature([], tf.string),
            'class': tf.io.FixedLenFeature([], tf.int64),
            'image': tf.io.FixedLenFeature([], tf.string)
            }
        self.image_dataset = tf.data.TFRecordDataset(self.image_files)
        self.image_dataset = self.image_dataset.map(self._parse_function)
        self.indexed_data = self._create_indexed_data(self.image_dataset)
        self.aug = albumentations.Compose([
            albumentations.Resize(228,228,always_apply=True),
            albumentations.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
    def _parse_function(self,example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
      return tf.io.parse_single_example(example_proto, feature_description)
    
    def _create_indexed_data(self,image_dataset):
        indexed_data = []
        for image_data in image_dataset:
            clazz = int(image_data['class'].numpy())
            id = str(image_data['id'].numpy())[2:-1]
            image = image_data['image'].numpy()
            indexed_data.append({'class':clazz,'id':id,'img':image})
        return indexed_data
    
    def __len__(self):
        return len(self.indexed_data)
        
    def __getitem__(self,index):
        item = self.indexed_data[index]
        img = item['img']
        img = np.array(Image.open(io.BytesIO(img)))
        img = self.aug(image = img)['image']
        img = np.transpose(img,(2,0,1)).astype(np.float32)
        img = torch.tensor(img,dtype=torch.float)
        return img,item['class']    
        

In [7]:
train_dataset = PetalDataset(train_files)
val_dataset=PetalDataset(val_files)

2022-03-18 20:37:23.292916: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-18 20:37:23.296505: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-03-18 20:37:23.296542: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-18 20:37:23.296568: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (67d53a6c0509): /proc/driver/nvidia/version does not exist
2022-03-18 20:37:23.299737: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [8]:
train_dl = DataLoader(train_dataset,batch_size=64,shuffle=True)
val_dl = DataLoader(val_dataset,batch_size=64,shuffle=True)

In [9]:
class PetalModel(nn.Module):
    def __init__(self,pre_trained=True):
        super(PetalModel,self).__init__()
        self.pre_trained=pre_trained
        self.model = models.resnet50(pretrained=self.pre_trained)
        self.model.fc = nn.Linear(self.model.fc.in_features,104)
    def forward(self,x):
        return self.model(x)

In [10]:
#device = torch.device('cuda')
device = xm.xla_device()

In [11]:
model = PetalModel()

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [12]:
params = {'lr':0.005, 
           'momentum':0.9, 
           'iterations':100}
optimizer = optim.SGD(model.parameters(),lr = params['lr'],momentum = params['momentum'])

In [13]:
run = neptune.init_project(name="njoysubho/petal", api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0NWQyMTAwNi0zMjc2LTRmMTktODQxNy1lZjdkZDA5ZjIyYmMifQ==")
run['parameters'] = params

https://app.neptune.ai/njoysubho/petal/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/project#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [14]:
model = model.to(device)

In [15]:
loss_func = nn.CrossEntropyLoss()

In [ ]:
for iteration in range(params['iterations']):
    accuracy = 0
    correct = 0
    val_correct = 0
    val_accuracy =0
    model.train()
    for (x,y) in train_dl:
        x,y = x.to(device),y.to(device)
        prediction = model(x)
        optimizer.zero_grad()
        train_loss = loss_func(prediction,y)
        _, predicted = torch.max(prediction.data, 1)
        correct += (predicted == y).float().sum()
        train_loss.backward()
        xm.optimizer_step(optimizer, barrier=True)
    accuracy = 100 * correct / len(train_dataset)
    run['metrics/train/accuracy'].log(accuracy)
    model.eval()
    with torch.no_grad():
        for xVal,yVal in val_dl:
            xVal,yVal= xVal.to(device),yVal.to(device)
            val_prediction = model(xVal)
            _, val_predicted = torch.max(val_prediction.data, 1)
            val_correct += (val_predicted == yVal).float().sum()
    val_accuracy = 100 * val_correct / len(val_dataset)
    run['metrics/val/accuracy'].log(val_accuracy)
    print("Epoch {} Train Accuracy = {} Validation Accuracy = {}".format(iteration,accuracy,val_accuracy))


Epoch 0 Train Accuracy = 5.253665924072266 Validation Accuracy = 5.387930870056152


In [ ]:
run.stop()

In [ ]:
#https://www.kaggle.com/code/piantic/pytorch-tpu/notebook
#https://medium.com/the-owl/installing-pytorch-xla-in-google-colab-without-35d37d1d03c6